In [1]:
import pandas as pd
from datetime import date

In [2]:
df = pd.read_csv('TMDB_movie_dataset_v11.csv')

In [3]:
df.sample(5)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
1159124,786565,The Return of Bruce Nauman’s Bouncing Balls,0.0,0,Released,2006-07-07,0,8,False,NaN,...,The Return of Bruce Nauman’s Bouncing Balls,"Conceived as an ironic tribute, the “Bruce Nau...",0.628,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68934,140172,Cool Air,4.8,12,Released,2006-11-03,0,88,False,NaN,...,Cool Air,"Charlie Baxter, a struggling screenwriter, is ...",1.514,/rtnli56H9xKUwtETeHxjUuuYaRU.jpg,DEATH COMES COLDLY,Horror,NaN,United States of America,NaN,NaN
484828,1142303,Mexico 2,0.0,0,Released,1961-01-01,0,13,False,NaN,...,Mexico 2,"Soundtrack: Manuel de Falla's HOMENAJE, POUR ...",0.600,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1018643,822962,Loiter,0.0,0,Released,2021-01-30,0,110,False,NaN,...,Loiter,Join us in the act of loitering around the Cha...,0.600,/pneMLBsdHOPKs0cIjVqq9R3DWo0.jpg,NaN,NaN,NaN,NaN,NaN,NaN
919253,243371,The Tide of Traffic,0.0,0,Released,1972-01-01,0,28,False,NaN,...,The Tide of Traffic,The Tide of Traffic is a 1972 British short do...,0.600,/vLvx2B54rOQ1OrmfrB5XJjU32et.jpg,NaN,Documentary,"Greenpark Productions, British Petroleum, Film...",United Kingdom,English,NaN


### Xóa các hàng không cần thiết:

In [4]:
df = df.drop(columns=['backdrop_path', 'homepage', 'imdb_id', 'tagline', 'keywords'])

### Lọc thời gian từ 1/1/2000 đến 31/3/2025:

In [5]:
# Chuyển đổi cột release_date sang kiểu datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_date'] = df['release_date'].dt.date

In [6]:
nat_ratio = df['release_date'].isna().mean()
print(f"Tỷ lệ các dòng có release_date là NaT: {nat_ratio:.2%}")

Tỷ lệ các dòng có release_date là NaT: 17.88%


In [7]:
df = df[(df['release_date'] >= date(2000, 1, 1)) & (df['release_date'] <= date(2025, 3, 31))]

In [8]:
df.sample(3)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,budget,original_language,original_title,overview,popularity,poster_path,genres,production_companies,production_countries,spoken_languages
17375,9391,Enduring Love,5.701,107,Released,2004-11-26,0,100,False,0,en,Enduring Love,Two strangers become dangerously close after w...,7.506,/ogaUARGgXWmNSuh8qnFCcbqVvZL.jpg,"Drama, Thriller, Mystery","Pathé Pictures International, UK Film Council,...",United Kingdom,English
1086794,929595,Womanhood,0.000,0,Released,2021-11-26,0,90,False,0,en,Womanhood,"In Womanhood, six famous women spanning differ...",0.600,NaN,NaN,NaN,NaN,English
23049,109786,Albakiara,2.200,68,Released,2008-10-28,0,93,False,0,it,Albakiara,Promiscuous teenager Chiara gets herself and h...,4.740,/DSAE8ZGTYJ9akUXc5PWKfNDADv.jpg,"Drama, Crime","Albachiara, Pixeldna, RAI Cinema, Mikado Film",Italy,Italian


### Chuyển định dạng dấu thập phân từ '.' sang ',':

In [9]:
df['vote_average'] = df['vote_average'].astype(str)
df['popularity'] = df['popularity'].astype(str)

In [10]:
df['vote_average'] = df['vote_average'].str.replace('.', ',', regex=False)
df['popularity'] = df['popularity'].str.replace('.', ',', regex=False)

### Lọc các phim có thời lượng trên 10 phút:

In [11]:
df = df[df['runtime'] > 10]

### Thêm cột quốc gia chính sản xuất phim:

In [12]:
df['primary_country'] = df['production_countries'].str.split(', ').str[0]

### Thêm cột công ty chính sản xuất phim:

In [13]:
df['primary_company'] = df['production_companies'].str.split(', ').str[0]

### Lọc các hàng có trạng thái là 'released':

In [14]:
df = df[df['status'] == 'Released']

### Genres:

In [15]:
# Tạo một set để lưu tất cả các thể loại (loại bỏ trùng lặp ngay từ đầu)
all_genres = set()

# Lặp qua từng giá trị trong cột genres
for genres in df['genres'].dropna():
    if ', ' in genres:  # Nếu có dấu phẩy, tách chuỗi
        all_genres.update(genres.split(', '))
    else:  # Nếu không có dấu phẩy, thêm trực tiếp
        all_genres.add(genres)

# Đếm số lượng thể loại riêng lẻ
num_genres = len(all_genres)

# In kết quả
print(f"Có {num_genres} thể loại riêng lẻ:")
print(all_genres)

Có 19 thể loại riêng lẻ:
{'Documentary', 'Western', 'Adventure', 'Thriller', 'Family', 'Fantasy', 'Science Fiction', 'Comedy', 'Music', 'Crime', 'Action', 'Drama', 'Animation', 'TV Movie', 'Horror', 'Mystery', 'History', 'War', 'Romance'}


---

In [16]:
df.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'budget', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'primary_country', 'primary_company'],
      dtype='object')

In [17]:
df.sample(3)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,budget,...,original_title,overview,popularity,poster_path,genres,production_companies,production_countries,spoken_languages,primary_country,primary_company
465604,1016382,White Trash Nurses,"0,0",0,Released,2009-03-18,0,125,True,0,...,White Trash Nurses,These sexy nurses are always on call and alway...,"0,6",/6tuorp0VUy2UzifkFQH52hg9gXx.jpg,NaN,Hustler Video,NaN,English,NaN,Hustler Video
687861,1211004,First Time Newbies,"0,0",0,Released,2023-06-05,0,205,True,0,...,First Time Newbies,Brand new 18-year-old teen newbie Xoey Li make...,"0,0",/diPaMbq7ky0NU7RsNtsKa9yGNYe.jpg,NaN,Hussie Pass,United States of America,English,United States of America,Hussie Pass
119411,55702,The Angel Doll,"5,5",5,Released,2002-09-14,0,93,False,0,...,The Angel Doll,The story of two small town boys from differen...,"1,839",/5yFZBl45d48LPpXVB6UVjonHC7I.jpg,"Drama, Family",NaN,United States of America,English,United States of America,NaN


In [18]:
df.shape

(422431, 21)

In [19]:
def missing_ratios(df):
    return df.isna().mean().sort_values(ascending=False)

In [20]:
missing_ratios(df)

primary_company         0.438547
production_companies    0.438547
primary_country         0.340349
production_countries    0.340349
genres                  0.318000
spoken_languages        0.293113
poster_path             0.157278
overview                0.033269
original_title          0.000000
popularity              0.000000
id                      0.000000
title                   0.000000
budget                  0.000000
adult                   0.000000
runtime                 0.000000
revenue                 0.000000
release_date            0.000000
status                  0.000000
vote_count              0.000000
vote_average            0.000000
original_language       0.000000
dtype: float64

### Lưu dữ liệu:

In [21]:
# Lưu DataFrame đã xử lý vào file CSV mới
df.to_csv('TMDB_movie_dataset_2000_2025.csv', index=False)

# Lưu ý:
- Cột `popularity` không rõ cách tính -> **không ưu tiên** trực quan cột này.
- Cột `genres` gồm nhiều thể loại trong một dòng, **chưa phân tách** mỗi thể loại thành mỗi cột vì sẽ làm tăng chiều dữ liệu -> tốn bộ nhớ -> giảm hiệu suất + lag máy.
- Cột `production_countries` gồm nhiều quốc gia trong một hàng -> đã thêm cột `primary_country` chỉ gồm một quốc gia, **có thể trực quan** cột này.
- Cột `production_companies` gồm nhiều công ty trong một hàng -> đã thêm cột `primary_company` chỉ gồm một công ty, **có thể trực quan** cột này.

---

In [2]:
df_preprocess = pd.read_csv('TMDB_movie_dataset_2000_2025.csv')

In [3]:
df_preprocess.shape

(422431, 21)

In [11]:
df_preprocess['id'].nunique()

422354

In [12]:
# Đếm số hàng trùng lặp trong df_preprocess
df_preprocess.duplicated().sum()

74

In [13]:
# Xóa các hàng trùng lặp trong df_preprocess
df_preprocess = df_preprocess.drop_duplicates()

In [16]:
df_preprocess.shape

(422357, 21)

In [17]:
# Tìm các giá trị id bị trùng lặp
duplicate_ids = df_preprocess[df_preprocess.duplicated(subset='id', keep=False)]

# Hiển thị các id bị trùng lặp
duplicate_id_values = duplicate_ids['id'].value_counts()

print("Các id bị trùng lặp và số lần xuất hiện:")
print(duplicate_id_values)

Các id bị trùng lặp và số lần xuất hiện:
id
1260267    2
1361654    2
1278956    2
Name: count, dtype: int64


In [82]:
# Chọn các dòng có id là 1260267
df_preprocess[df_preprocess['id'] == 1278956]

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,budget,...,original_title,overview,popularity,poster_path,genres,production_companies,production_countries,spoken_languages,primary_country,primary_company
266116,1278956,Estancia,"0,0",0,Released,2024-04-20,0,85,False,0,...,Estancia,"In the chaotic and seedy heart of Medellín, th...","1,4",NaN,NaN,NaN,Colombia,NaN,Colombia,NaN


In [75]:
df_preprocess = df_preprocess.drop(133224)

In [78]:
df_preprocess = df_preprocess.drop(236582)

In [81]:
df_preprocess = df_preprocess.drop(266117)

In [83]:
# xóa cột primary_company & primary_country
df_preprocess = df_preprocess.drop(columns=['primary_company', 'primary_country'])

In [84]:
df_preprocess.head(3)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,budget,original_language,original_title,overview,popularity,poster_path,genres,production_companies,production_countries,spoken_languages
0,27205,Inception,"8,364",34495,Released,2010-07-15,825532764,148,False,160000000,en,Inception,"Cobb, a skilled thief who commits corporate es...","83,952",/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili"
1,157336,Interstellar,"8,417",32571,Released,2014-11-05,701729206,169,False,165000000,en,Interstellar,The adventures of a group of explorers who mak...,"140,241",/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English
2,155,The Dark Knight,"8,512",30619,Released,2008-07-16,1004558444,152,False,185000000,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,"130,643",/qJ2tW6WMUDux911r6m7haRef0WH.jpg,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin"


In [85]:
df_preprocess.shape

(422354, 19)

In [86]:
df_preprocess.to_csv('TMDB_movie_dataset_2000_2025.csv', index=False)

---

In [5]:
df_genres = pd.read_csv('movie_genre_table.csv')

In [30]:
df_genres.head(3)

,Unnamed: 0,id,Action,Science Fiction,Adventure,Drama,Crime,Thriller,Fantasy,Comedy,...,Mystery,War,Animation,Family,Horror,Music,History,Romance,TV Movie,Documentary
0,0,27205,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,157336,False,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2,155,True,False,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
df_genres.shape

(422431, 21)

In [44]:
df_genres['id'].nunique()

422354

In [31]:
# Xóa cột Unnamed:
df_genres = df_genres.drop(columns=['Unnamed: 0'])

In [43]:
df_genres.duplicated().sum()

0

In [33]:
df_genres = df_genres.drop_duplicates()

In [42]:
df_genres.shape

(422354, 20)

In [41]:
'''Các id bị trùng lặp và số lần xuất hiện:
id
1260267    2
1361654    2
1278956    2
Name: count, dtype: int64'''
df_genres[df_genres['id'] == 1260267]

,id,Action,Science Fiction,Adventure,Drama,Crime,Thriller,Fantasy,Comedy,Western,Mystery,War,Animation,Family,Horror,Music,History,Romance,TV Movie,Documentary
133223,1260267,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [45]:
df_genres.to_csv('movie_genre_table.csv', index=False)

---

In [46]:
df_companies = pd.read_csv('production_companies.csv')

C:\Users\ROCK\AppData\Local\Temp\ipykernel_21324\3801725380.py:1: DtypeWarning: Columns (11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df_companies = pd.read_csv('production_companies.csv')


In [47]:
df_companies.shape

(422431, 32)

In [49]:
df_companies['id'].nunique()

422354

In [50]:
df_companies = df_companies.drop(columns=['Unnamed: 0'])

In [51]:
df_companies.duplicated().sum()

77

In [52]:
df_companies = df_companies.drop_duplicates()

In [53]:
df_companies.shape

(422354, 31)

In [58]:
df_companies[df_companies['id'] == 1278956]

,id,product_company_n1,product_company_n2,product_company_n3,product_company_n4,product_company_n5,product_company_n6,product_company_n7,product_company_n8,product_company_n9,...,product_company_n21,product_company_n22,product_company_n23,product_company_n24,product_company_n25,product_company_n26,product_company_n27,product_company_n28,product_company_n29,product_company_n30
266116,1278956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df_companies.to_csv('production_companies.csv', index=False)

---

In [60]:
df_countries = pd.read_csv('production_countries.csv')

C:\Users\ROCK\AppData\Local\Temp\ipykernel_21324\187581462.py:1: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df_countries = pd.read_csv('production_countries.csv')


In [61]:
df_countries.shape

(422431, 61)

In [63]:
df_countries['id'].nunique()

422354

In [64]:
df_countries = df_countries.drop(columns=['Unnamed: 0'])

In [65]:
df_countries.duplicated().sum()

77

In [66]:
df_countries = df_countries.drop_duplicates()

In [67]:
df_countries.shape

(422354, 60)

In [71]:
df_countries[df_countries['id'] == 1260267]

,id,product_country_n1,product_country_n2,product_country_n3,product_country_n4,product_country_n5,product_country_n6,product_country_n7,product_country_n8,product_country_n9,...,product_country_n50,product_country_n51,product_country_n52,product_country_n53,product_country_n54,product_country_n55,product_country_n56,product_country_n57,product_country_n58,product_country_n59
133223,1260267,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
df_countries.to_csv('production_countries.csv', index=False)